In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Goiás - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Goiás - Consumo de Cimento (t)
0,2003-1,56.459960,0.745293,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.260097,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,8.179785,97.447
1,2003-2,56.789152,0.745405,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.258964,9.383590e+07,5.523290e+06,14.034476,8.422082e+07,8.174086,97.256
2,2003-3,57.140263,0.745517,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.257352,9.395358e+07,5.528254e+06,14.037682,8.425916e+07,8.168386,101.442
3,2003-4,57.649192,0.745629,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.255929,9.407126e+07,5.533219e+06,14.040888,8.429750e+07,8.162687,98.217
4,2003-5,58.213295,0.745741,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.254109,9.418894e+07,5.538184e+06,14.044093,8.433584e+07,8.156988,112.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,175.674487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,274.673
236,2022-9,174.614843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,247.240
237,2022-10,173.307223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,246.647
238,2022-11,171.556538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,225.813


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503
188,1.677410,0.682578,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711
189,1.732332,0.638983,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918
190,1.763870,0.595389,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Goiás - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.623245,1.561651,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.944351,1.175698,-1.121085,-0.108013,0.955652,1.757990
158,0.644558,1.540971,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.952621,1.178649,-1.137848,-0.116498,0.956673,1.696610
159,0.663382,1.520292,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.960982,1.181601,-1.154611,-0.124984,0.957693,1.635230
160,0.670170,1.499612,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.969363,1.184552,-1.171374,-0.133469,0.958714,1.573850


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
157    146.310
158    191.374
159    164.318
160    211.049
161    198.789
Name: Goiás - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
162,0.739555,1.458252,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,0.985437,1.190455,-1.204899,-0.150440,0.960755,1.451090
163,0.782153,1.437573,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,0.993928,1.193406,-1.221662,-0.158926,0.961776,1.389710
164,0.818056,1.416893,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.002369,1.196357,-1.238425,-0.167411,0.962797,1.328330
165,0.858655,1.396213,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.010759,1.199309,-1.255188,-0.175897,0.963817,1.266951
166,0.883743,1.375533,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.019096,1.202260,-1.271951,-0.184382,0.964838,1.205571
167,0.897762,1.354854,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.027156,1.205211,-1.288714,-0.192868,0.965858,1.144191
168,0.920054,1.334174,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.039065,1.208162,-1.305477,-0.201353,0.966879,1.082811
169,0.936187,1.308937,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.053773,1.204916,-1.303958,-0.221752,0.959012,1.073414
170,0.983823,1.283700,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.068604,1.201670,-1.302439,-0.242150,0.951146,1.064018
171,1.015815,1.258463,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.079870,1.198423,-1.300920,-0.262549,0.943279,1.054621


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    205.551
163    230.558
164    209.621
165    210.984
166    189.889
167    143.721
168    176.275
169    152.632
170    177.876
171    181.384
172    160.120
173    236.951
174    206.557
175    224.270
176    213.871
177    211.066
178    182.899
179    159.002
180    178.918
181    161.936
182    173.792
183    176.627
184    207.991
185    185.784
186    234.176
187    226.996
188    218.141
189    234.939
190    199.619
191    168.797
Name: Goiás - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3815968419, 3616126460, 2450917004, 3170319086, 807354513, 3485468233, 2269354474, 2021896228, 885106016, 3644900699, 3932650742, 3441310037, 3471915148, 789505004, 4188518178, 1506224309, 2658187861, 3231072662, 1865664446, 2102788858, 3877007843, 2325476664, 2792590970, 1951573899, 200638365]


Step: 0 ___________________________________________
val_loss: 483.4457092285156
winner_seed: 3815968419


Step: 1 ___________________________________________
val_loss: 438.1943054199219
winner_seed: 3616126460


Step: 2 ___________________________________________
val_loss: 352.896728515625
winner_seed: 2450917004


Step: 3 ___________________________________________
val_loss: 377.7671813964844


Step: 4 ___________________________________________
val_loss: 492.96844482421875


Step: 5 ___________________________________________
val_loss: 697.4258422851562


Step: 6 ___________________________________________
val_loss: 395.3368835449219


Step: 7 ___________________________________________
val

2023-10-14 17:03:41.232552: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 440.8505859375


Step: 20 ___________________________________________
val_loss: 1192.569091796875


Step: 21 ___________________________________________
val_loss: 501.41717529296875


Step: 22 ___________________________________________
val_loss: 431.1412048339844


Step: 23 ___________________________________________
val_loss: 389.2601013183594


Step: 24 ___________________________________________
val_loss: 441.0576477050781


final_seed: 2450917004


In [18]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 25ms/step - loss: 40747.6680 - val_loss: 31797.7559
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 23732.8125 - val_loss: 7300.6875
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 24339.9082 - val_loss: 26391.3867
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 25054.0352 - val_loss: 23211.1406
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 15241.7090 - val_loss: 14378.2461
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 7791.2480 - val_loss: 1349.6625
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 904.6514 - val_loss: 916.7559
Epoch 8/10000
5/5 [==============================] - 0s 6ms/step - loss: 736.0385 - val_loss: 1523.1346
Epoch 9/10000
5/5 [==============================] - 0s 6ms/step - loss: 892.2529 - val_loss: 752.2831
Epoch 10/10000
5/5 [==============================

5/5 [==============================] - 0s 5ms/step - loss: 650.8027 - val_loss: 847.6255
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 635.7880 - val_loss: 730.3021
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 580.6917 - val_loss: 685.3934
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 524.2654 - val_loss: 661.3067
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 559.1360 - val_loss: 734.7274
Epoch 84/10000
5/5 [==============================] - 0s 6ms/step - loss: 585.5134 - val_loss: 756.7822
Epoch 85/10000
5/5 [==============================] - 0s 6ms/step - loss: 553.0947 - val_loss: 877.5190
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 583.0309 - val_loss: 862.1359
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 647.9034 - val_loss: 769.9813
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 643.0

5/5 [==============================] - 0s 6ms/step - loss: 558.7000 - val_loss: 738.0422
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 579.5875 - val_loss: 642.7311
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 499.1586 - val_loss: 837.5447
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 548.4983 - val_loss: 675.4780
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 517.7284 - val_loss: 586.9794
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 488.0645 - val_loss: 571.4058
Epoch 163/10000
5/5 [==============================] - 0s 6ms/step - loss: 497.9703 - val_loss: 691.3889
Epoch 164/10000
5/5 [==============================] - 0s 6ms/step - loss: 549.6462 - val_loss: 810.2402
Epoch 165/10000
5/5 [==============================] - 0s 6ms/step - loss: 562.0230 - val_loss: 751.8254
Epoch 166/10000
5/5 [==============================] - 0s 6ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 570.7079 - val_loss: 620.0151
Epoch 236/10000
5/5 [==============================] - 0s 6ms/step - loss: 519.5690 - val_loss: 547.4739
Epoch 237/10000
5/5 [==============================] - 0s 6ms/step - loss: 480.2008 - val_loss: 845.1248
Epoch 238/10000
5/5 [==============================] - 0s 6ms/step - loss: 566.4881 - val_loss: 711.9320
Epoch 239/10000
5/5 [==============================] - 0s 6ms/step - loss: 504.1777 - val_loss: 698.4907
Epoch 240/10000
5/5 [==============================] - 0s 6ms/step - loss: 519.4407 - val_loss: 789.6591
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 557.2155 - val_loss: 812.8763
Epoch 242/10000
5/5 [==============================] - 0s 6ms/step - loss: 463.2988 - val_loss: 573.0234
Epoch 243/10000
5/5 [==============================] - 0s 6ms/step - loss: 450.8521 - val_loss: 555.3705
Epoch 244/10000
5/5 [==============================] - 0s 6ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 639.2510 - val_loss: 840.6177
Epoch 314/10000
5/5 [==============================] - 0s 6ms/step - loss: 641.0722 - val_loss: 823.6422
Epoch 315/10000
5/5 [==============================] - 0s 6ms/step - loss: 565.9092 - val_loss: 780.2116
Epoch 316/10000
5/5 [==============================] - 0s 6ms/step - loss: 582.3522 - val_loss: 767.8047
Epoch 317/10000
5/5 [==============================] - 0s 6ms/step - loss: 625.7787 - val_loss: 881.6002
Epoch 318/10000
5/5 [==============================] - 0s 6ms/step - loss: 630.0757 - val_loss: 714.1630
Epoch 319/10000
5/5 [==============================] - 0s 6ms/step - loss: 620.7424 - val_loss: 735.8222
Epoch 320/10000
5/5 [==============================] - 0s 5ms/step - loss: 597.5443 - val_loss: 743.8989
Epoch 321/10000
5/5 [==============================] - 0s 6ms/step - loss: 519.6903 - val_loss: 687.6613
Epoch 322/10000
5/5 [==============================] - 0s 6ms/step - lo

...

5/5 [==============================] - 0s 8ms/step - loss: 523.2699 - val_loss: 803.1594
Epoch 1952/10000
5/5 [==============================] - 0s 6ms/step - loss: 492.8893 - val_loss: 714.6265
Epoch 1953/10000
5/5 [==============================] - 0s 7ms/step - loss: 471.1339 - val_loss: 775.9288
Epoch 1954/10000
5/5 [==============================] - 0s 6ms/step - loss: 464.2545 - val_loss: 825.6035
Epoch 1955/10000
5/5 [==============================] - 0s 7ms/step - loss: 437.7782 - val_loss: 704.1725
Epoch 1956/10000
5/5 [==============================] - 0s 6ms/step - loss: 432.4146 - val_loss: 642.9688
Epoch 1957/10000
5/5 [==============================] - 0s 6ms/step - loss: 422.7674 - val_loss: 633.7578
Epoch 1958/10000
5/5 [==============================] - 0s 7ms/step - loss: 475.6791 - val_loss: 703.8309
Epoch 1959/10000
5/5 [==============================] - 0s 6ms/step - loss: 436.9984 - val_loss: 766.4846
Epoch 1960/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 7ms/step - loss: 459.3079 - val_loss: 536.7735
Epoch 2029/10000
5/5 [==============================] - 0s 7ms/step - loss: 455.1803 - val_loss: 800.9725
Epoch 2030/10000
5/5 [==============================] - 0s 7ms/step - loss: 535.1907 - val_loss: 584.3554
Epoch 2031/10000
5/5 [==============================] - 0s 7ms/step - loss: 441.0721 - val_loss: 541.0146
Epoch 2032/10000
5/5 [==============================] - 0s 7ms/step - loss: 366.2483 - val_loss: 488.9573
Epoch 2033/10000
5/5 [==============================] - 0s 7ms/step - loss: 409.6614 - val_loss: 490.9439
Epoch 2034/10000
5/5 [==============================] - 0s 7ms/step - loss: 405.6005 - val_loss: 622.1743
Epoch 2035/10000
5/5 [==============================] - 0s 8ms/step - loss: 405.7101 - val_loss: 540.2714
Epoch 2036/10000
5/5 [==============================] - 0s 7ms/step - loss: 459.7760 - val_loss: 522.6813
Epoch 2037/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 7ms/step - loss: 451.9611 - val_loss: 468.8497
Epoch 2106/10000
5/5 [==============================] - 0s 7ms/step - loss: 434.1296 - val_loss: 577.1214
Epoch 2107/10000
5/5 [==============================] - 0s 7ms/step - loss: 525.2937 - val_loss: 435.7539
Epoch 2108/10000
5/5 [==============================] - 0s 7ms/step - loss: 1599.6798 - val_loss: 836.9987
Epoch 2109/10000
5/5 [==============================] - 0s 6ms/step - loss: 713.6340 - val_loss: 1018.8455
Epoch 2110/10000
5/5 [==============================] - 0s 7ms/step - loss: 572.0183 - val_loss: 618.7432
Epoch 2111/10000
5/5 [==============================] - 0s 7ms/step - loss: 487.6343 - val_loss: 569.4642
Epoch 2112/10000
5/5 [==============================] - 0s 8ms/step - loss: 444.2708 - val_loss: 591.6307
Epoch 2113/10000
5/5 [==============================] - 0s 8ms/step - loss: 522.1154 - val_loss: 576.7653
Epoch 2114/10000
5/5 [==============================] - 0s 7m

5/5 [==============================] - 0s 7ms/step - loss: 574.0016 - val_loss: 551.6078
Epoch 2183/10000
5/5 [==============================] - 0s 7ms/step - loss: 522.5136 - val_loss: 509.3096
Epoch 2184/10000
5/5 [==============================] - 0s 6ms/step - loss: 592.4037 - val_loss: 490.4471
Epoch 2185/10000
5/5 [==============================] - 0s 6ms/step - loss: 452.0985 - val_loss: 554.1722
Epoch 2186/10000
5/5 [==============================] - 0s 6ms/step - loss: 449.4845 - val_loss: 965.1905
Epoch 2187/10000
5/5 [==============================] - 0s 6ms/step - loss: 551.5774 - val_loss: 641.1449
Epoch 2188/10000
5/5 [==============================] - 0s 6ms/step - loss: 492.6340 - val_loss: 486.4727
Epoch 2189/10000
5/5 [==============================] - 0s 6ms/step - loss: 424.5481 - val_loss: 598.2297
Epoch 2190/10000
5/5 [==============================] - 0s 6ms/step - loss: 469.0018 - val_loss: 481.0736
Epoch 2191/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 6ms/step - loss: 529.0366 - val_loss: 633.5953
Epoch 2260/10000
5/5 [==============================] - 0s 7ms/step - loss: 477.5634 - val_loss: 609.5405
Epoch 2261/10000
5/5 [==============================] - 0s 8ms/step - loss: 421.4673 - val_loss: 779.8411
Epoch 2262/10000
5/5 [==============================] - 0s 7ms/step - loss: 474.8051 - val_loss: 757.0441
Epoch 2263/10000
5/5 [==============================] - 0s 7ms/step - loss: 450.8898 - val_loss: 773.1549
Epoch 2264/10000
5/5 [==============================] - 0s 6ms/step - loss: 467.1741 - val_loss: 785.8568
Epoch 2265/10000
5/5 [==============================] - 0s 6ms/step - loss: 457.4198 - val_loss: 839.6348
Epoch 2266/10000
5/5 [==============================] - 0s 8ms/step - loss: 519.5814 - val_loss: 922.7690
Epoch 2267/10000
5/5 [==============================] - 0s 8ms/step - loss: 505.7433 - val_loss: 943.8286
Epoch 2268/10000
5/5 [==============================] - 0s 6ms/

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,195.550018,177.931335,133.576996,-8.074715,-40.573006,-40.580818,-40.580818,-40.580818,-40.580818
Target,205.551,230.558,209.621,210.984,189.889,143.721,176.275,152.632,177.876,181.384,160.12,236.951,206.557,224.27,213.871,211.066,182.899,159.002,178.918,161.936,173.792,176.627,207.991,185.784,234.176,226.996,218.141,234.939,199.619,168.797
Error,10.000977,35.00798,14.070984,15.433975,5.661011,51.829025,19.275024,42.918015,17.674011,14.166016,35.430023,41.400986,11.006989,28.719986,18.320984,15.515976,12.651016,36.548019,16.632019,33.614014,21.758011,18.923019,30.059662,52.207001,242.250702,267.569,258.721832,275.519806,240.199829,209.377808


In [21]:
display(mae)
display(mape)

69.74879

0.34362772

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[2346.6003]] - Target[2275.5620000000004]| =  Error: [[71.03833]]; MAPE:[[0.03121793]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[2267.0085]] - Target[2282.713]| =  Error: [[15.704346]]; MAPE:[[0.00687968]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[-210.97098]] - Target[1282.668]| =  Error: [[1493.6389]]; MAPE:[[1.1644782]]


[array([[71.03833]], dtype=float32),
 array([[15.704346]], dtype=float32),
 array([[1493.6389]], dtype=float32)]

526.7939

0.4008586